In [66]:
import os
import pandas as pd
import numpy as np
import pickle
import PIL
from PIL import Image
import matplotlib.pyplot as plt

#import custom functions
from image_load import load_images, get_image_stats
from image_transform import pad_images, crop_images

In [65]:
test_fire_images = load_images('corsican_images',1,100)
test_nonfire_images = load_images('kaggle_landscape_images',0,100)

In [70]:
data = []
#image types 
smokeTypes = ['Heavy Smoke','Light Smoke']
timeOfDay = ['Night','Day','Morning']

# for pulling non-fire photos
nofire_processed = load_images('wildfire_train_set - Base Forest - Processed',0,100)
data.extend(nofire_processed)

# for pulling fire photos
for smokeType in smokeTypes:
    for time in timeOfDay:
        wildfire_train_set_Processed = load_images(f'wildfire_train_set - Processed and Pruned/{smokeType}/{time}', 1, 100)
        data.extend(wildfire_train_set_Processed)

images, labels = zip(*data)
length_mean, length_max, width_max, width_mean = get_image_stats(images)
#crop and pad images
images_cropped = pad_images(images, width_max, length_max)
images_final = crop_images(images_cropped, width_mean, length_mean)
#convert all imagees to RGB some are ARGB
images_final = [np.array(Image.fromarray(image).convert('RGB')) for image in images_final]
#add labels back in
data_train = list(zip(images_final, labels))

In [73]:
data_test = []
data_test.extend(test_fire_images[:45])
data_test.extend(test_nonfire_images)
images_test, labels_test = zip(*data_test)

images_final_test = []
for img in images_test:
    img = Image.fromarray(img)
    img = img.resize((177, 100))
    images_final_test.append(np.array(img))

data_test = list(zip(images_final_test, labels_test))

In [74]:
output_file1 = 'train_images_pkl.pkl'
output_file2 = 'test_images_pkl.pkl'

with open('{}/artifacts/{}'.format(dir,output_file1), 'wb+') as out:
    pickle.dump(data_train, out)

with open('{}/artifacts/{}'.format(dir,output_file2), 'wb+') as out:
     pickle.dump(data_test, out)